# Logistic Regression with Newton's Method to predict if someone has diabetes or not

### Importing Dependencies

In [1]:
# Matrix Math
import numpy as np

# Data Manipulation
import pandas as pd

# Matrix data structure
from patsy import dmatrices

# for error logging
import warnings

### Setup: Parameters and data setup

#### Sigmoid function

$$y = \frac{\mathrm{1}}{\mathrm{1} + e^{-x}}$$

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

#### Data Preparation

In [3]:
# set the seed
np.random.seed(0)

# Convergance tolerance
tol = 1e-8

# Max iterations
max_iterations = 20

# data creation settings
r = 0.95 # covariance between x and z
n = 1000 # no of observations
sigma = 1 # variance of noise (i.e. how spread our data is)

# Model settings
beta_height, beta_weight, beta_blood_pressure = -4, 9, 1 # true beta coefficients
var_height, var_weight, var_blood_pressure = 1, 1, 4 # variances of inputs

# the model
formula = 'y ~ height + weight + blood_pressure + np.exp(height) + I(blood_pressure**2 + weight)'

# Data generation
# Keeping height and weight closly related
height, weight = np.random.multivariate_normal([0,0], [[var_height, r],[r, var_weight]], n).T
# blood pressure
blood_pressure = np.random.normal(0, var_blood_pressure, n)**3

# Creating dataframe
df = pd.DataFrame({'height': height, 'weight': weight, 'blood_pressure': blood_pressure})

df.head()


,blood_pressure,height,weight
0,-230.536312,-1.805133,-1.678592
1,-321.120883,-1.320743,-0.612110
2,0.006285,-1.689545,-1.998587
3,-56.335960,-0.914205,-0.962069
4,-0.033775,0.036999,0.166842


#### Log odds or Logits of the independent variables

for,

$$ \beta_{0} + \beta_1 x_1 + \beta_1 x_1 + \dots + \beta_n x_n $$

General Model,

$$ logit(P_{disease}) = \log{\bigg(\frac{P_{disease}}{1 - P_{disease}}\bigg)} = \beta_{0} + \beta_1 x_1 + \beta_1 x_1 + \dots + \beta_n x_n $$

logistics function:

$$ \operatorname{Pr} = \frac{\exp(\beta_{0} + \beta_{1} x_1 + \beta_{2} x_2 + \dots + \beta_{n} x_n)} {1 + \exp(\beta_{0} + \beta_{1} x_1 + \beta_{2} x_2 + \dots + \beta_{n} x_n)} \label{eq:glm1} $$

In [4]:
# Computing log odds of 3 independent variable
df['log_odds'] = sigmoid(sigma*np.random.normal(0,1,n) + df[['height', 'weight', 'blood_pressure']].dot([beta_height, beta_weight, beta_blood_pressure]))

df.head()

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


,blood_pressure,height,weight,log_odds
0,-230.536312,-1.805133,-1.678592,1.399942e-103
1,-321.120883,-1.320743,-0.612110,4.873115e-140
2,0.006285,-1.689545,-1.998587,1.191969e-05
3,-56.335960,-0.914205,-0.962069,2.955508e-27
4,-0.033775,0.036999,0.166842,5.272102e-01


In [5]:
# Computing probability sample from binomial distribution
df['y'] = [np.random.binomial(1, p) for p in df.log_odds]

df.head()

,blood_pressure,height,weight,log_odds,y
0,-230.536312,-1.805133,-1.678592,1.399942e-103,0
1,-321.120883,-1.320743,-0.612110,4.873115e-140,0
2,0.006285,-1.689545,-1.998587,1.191969e-05,0
3,-56.335960,-0.914205,-0.962069,2.955508e-27,0
4,-0.033775,0.036999,0.166842,5.272102e-01,0


In [8]:
# creating dataframe that encomapasses out input data, model formula and outputs
y, x = dmatrices(formula, df, return_type='dataframe')

x.head()

,Intercept,height,weight,blood_pressure,np.exp(height),I(blood_pressure ** 2 + weight)
0,1.0,-1.805133,-1.678592,-230.536312,0.164453,53145.312449
1,1.0,-1.320743,-0.612110,-321.120883,0.266937,103118.009125
2,1.0,-1.689545,-1.998587,0.006285,0.184604,-1.998547
3,1.0,-0.914205,-0.962069,-56.335960,0.400835,3172.778273
4,1.0,0.036999,0.166842,-0.033775,1.037692,0.167983


### Algorithm Setup

#### Explanation of a Single Newton Step

Newton's method for maximizing / minimizing a given function $f(\beta)$ iteratively computes the following estimate:

$$
\beta^+ = \beta - Hf(\beta)^{-1}\nabla f(\beta)
$$

The Hessian of the log-likelihood for logistic regression is given by:

hessian of our function = **X => negative tranpose of (N times (p+1))** times **W => (N x N diagional matrix of weights, each is p*(1-p))** times **X** again

$$
Hf(\beta) = -X^TWX
$$
and the gradient is:

gradient of our function = **tranpose of X** times (**column vector** - **N vector of probabilities**)

$$
\nabla f(\beta) = X^T(y-p)
$$
where $$
W := \text{diag}\left(p(1-p)\right)
$$ and $p$ are the predicted probabilites computed at the current value of $\beta$.

In [18]:
def newtons_method_steps(current_beta, X, learning_rate=0.0001):
    # create probability matrix, miniminum 2 dimensions, tranpose (flip it)
    p = np.array(sigmoid(X.dot(current_beta[:,0])), ndmin=2).T
    # create weight matrix
    W = np.diag((p * (1-p))[:,0])
    # derive the hessian 
    hessian = X.T.dot(W).dot(X)
    # derive the gradient
    gradient = X.T.dot(y-p)
    
    # invert of Hessian
    hessian = np.linalg.inv(hessian)
    
    step = learning_rate * np.matmul(hessian, gradient)
    
    # updated beta
    return current_beta + step

### Training

In [21]:
def is_coefs_converged(old_beta, new_beta, current_iterations, tolerance = 1e-8):
    # calculate the change in the coefficients
    coef_change = np.abs(old_beta - new_beta)
    
    # if change hasn't reached the threshold and we have more iterations to go, keep training
    return not (np.any(coef_change > tolerance) & (current_iterations < max_iterations))

In [23]:
## initial conditions
# initial coefficients (weight values), 2 copies, we'll update one
beta_old, beta = np.ones((len(x.columns),1)), np.zeros((len(x.columns),1))

# num iterations we've done so far
iteration_count = 0

# have we reached convergence?
coefs_converged = False

# if we haven't reached convergence... (keep training)
while not coefs_converged:
    # set the old coefficients to our current
    beta_old = beta
    
    # perform a single step of newton's optimization on our data, set our updated beta values
    beta = newtons_method_steps(beta, x)
    
    # increment the number of iterations
    iteration_count += 1
    
    # check for convergence between our old and new beta values
    coefs_converged = is_coefs_converged(beta_old, beta, iteration_count, tol)
    
print('Iterations : {}'.format(iteration_count))
print('Beta : {}'.format(beta))

Iterations : 20
Beta : [[-4.16126038e-05]
 [-9.19873632e-04]
 [ 2.02663642e-03]
 [ 9.85558530e-06]
 [-3.32659974e-05]
 [ 2.32494397e-09]]


In [ ]:
def predict(height, weight, blood_pressure):
    return 